Space X advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars; other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. Therefore if we can determine if the first stage will land, we can determine the cost of a launch. This information can be used if an alternate company wants to bid against space X for a rocket launch. In this lab, you will create a machine learning pipeline to predict if the first stage will land given the data from the preceding labs.

Perform exploratory Data Analysis and determine Training Labels

create a column for the class
Standardize the data
Split into training data and test data
-Find best Hyperparameter for SVM, Classification Trees and Logistic Regression

Find the method performs best using test data

In [ ]:
# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
# NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np
# Matplotlib is a plotting library for python and pyplot gives us a MatLab like plotting framework. We will use this in our plotter function to plot data.
import matplotlib.pyplot as plt
#Seaborn is a Python data visualization library based on matplotlib. It provides a high-level interface for drawing attractive and informative statistical graphics
import seaborn as sns
# Preprocessing allows us to standarsize our data
from sklearn import preprocessing
# Allows us to split our data into training and testing data
from sklearn.model_selection import train_test_split
# Allows us to test parameters of classification algorithms and find the best one
from sklearn.model_selection import GridSearchCV
# Logistic Regression classification algorithm
from sklearn.linear_model import LogisticRegression
# Support Vector Machine classification algorithm
from sklearn.svm import SVC
# Decision Tree classification algorithm
from sklearn.tree import DecisionTreeClassifier
# K Nearest Neighbors classification algorithm
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
def plot_confusion_matrix(y,y_predict):
    "this function plots the confusion matrix"
    from sklearn.metrics import confusion_matrix

    cm = confusion_matrix(y, y_predict)
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax); #annot=True to annotate cells
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels')
    ax.set_title('Confusion Matrix');
    ax.xaxis.set_ticklabels(['did not land', 'land']); ax.yaxis.set_ticklabels(['did not land', 'landed'])
    plt.show()

Load the dataframe

In [ ]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_2.csv"
URL2 = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_3.csv'
df = pd.read_csv(url)

# verify successful load with some randomly selected records
df.sample(5)

FlightNumber        Date BoosterVersion  PayloadMass Orbit    LaunchSite  \
68            69  2019-11-11       Falcon 9      15600.0  VLEO  CCAFS SLC 40   
61            62  2019-01-11       Falcon 9       9600.0    PO   VAFB SLC 4E   
71            72  2020-01-07       Falcon 9      15400.0  VLEO  CCAFS SLC 40   
7              8  2014-07-14       Falcon 9       1316.0   LEO  CCAFS SLC 40   
76            77  2020-03-18       Falcon 9      15600.0  VLEO    KSC LC 39A   

       Outcome  Flights  GridFins  Reused  Legs                LandingPad  \
68   True ASDS        4      True    True  True  5e9e3032383ecb6bb234e7ca   
61   True ASDS        2      True    True  True  5e9e3033383ecbb9e534e7cc   
71   True ASDS        4      True    True  True  5e9e3032383ecb6bb234e7ca   
7   True Ocean        1     False   False  True                       NaN   
76  False ASDS        5      True    True  True  5e9e3032383ecb6bb234e7ca   

    Block  ReusedCount Serial   Longitude   Latitude  Class  
68    5.0            4  B1048  -80.577366  28.561857      1  
61    5.0            5  B1049 -120.610829  34.632093      1  
71    5.0            5  B1049  -80.577366  28.561857      1  
7     1.0            0  B1007  -80.577366  28.561857      1  
76    5.0            4  B1048  -80.603956  28.608058      0

TASK 1
Create a NumPy array from the column Class in data, by applying the method to_numpy() then assign it to the variable Y,make sure the output is a Pandas series (only one bracket df['name of column']).

In [ ]:
# Estrarre colonna Class come Pandas Series e convertirla in NumPy array
Y = df['Class'].to_numpy()

# Controllo
print(type(Y))   # <class 'numpy.ndarray'>
print(Y[:10])    # mostra i primi 10 valori

<class 'numpy.ndarray'>
[0 0 0 0 0 0 1 1 0 0]


TASK 2
Standardize the data in X then reassign it to the variable X using the transform provided below.

We split the data into training and testing data using the function train_test_split. The training data is divided into validation data, a second set used for training data; then the models are trained and hyperparameters are selected using the function GridSearchCV

In [ ]:
#TASK 1 2 3
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# Carico il dataset
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_2.csv"
data = pd.read_csv(url)

# ============ TASK 1 ============
# Creare un array NumPy dalla colonna 'Class'
Y = data['Class'].to_numpy()

# ============ TASK 2 ============
# Selezioniamo solo le colonne numeriche
X = data.drop('Class', axis=1)
X = X.select_dtypes(include=['float64', 'int64'])

# Standardizzazione
transform = preprocessing.StandardScaler()
X = transform.fit_transform(X)


# ============ TASK 3 ============
# Split train/test con test_size=0.2 e random_state=2
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

# Controllo delle dimensioni
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("Y_train shape:", Y_train.shape)
print("Y_test shape:", Y_test.shape)


X_train shape: (72, 7)
X_test shape: (18, 7)
Y_train shape: (72,)
Y_test shape: (18,)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# ============ TASK 4 ============

# Definiamo i parametri da testare
parameters = {'C':[0.01,0.1,1],
              'penalty':['l2'],
              'solver':['lbfgs']}

# Creiamo il modello di regressione logistica
lr = LogisticRegression(max_iter=1000)  # max_iter alzato per evitare warning di convergenza

# Creiamo il GridSearchCV con 10-fold cross-validation
logreg_cv = GridSearchCV(lr, parameters, cv=10)

# Fit sul training set
logreg_cv.fit(X_train, Y_train)

# Migliori parametri trovati
print("Best parameters:", logreg_cv.best_params_)

# Accuracy migliore
print("Best score:", logreg_cv.best_score_)

# Valutazione sul test set
print("Test set accuracy:", logreg_cv.score(X_test, Y_test))


Best parameters: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
Best score: 0.7446428571428572
Test set accuracy: 0.8333333333333334


In [ ]:
# ============ TASK 5 ============

# Calcolo dell'accuracy sul test set
test_accuracy = logreg_cv.score(X_test, Y_test)

print("Accuracy on test data:", test_accuracy)


Accuracy on test data: 0.8333333333333334


In [ ]:
from sklearn.svm import SVC

# ============ TASK 6 ============

# Dizionario dei parametri da testare
parameters = {
    'kernel': ('linear', 'rbf', 'poly', 'sigmoid'),  # tolgo il duplicato 'rbf'
    'C': np.logspace(-3, 3, 5),   # [0.001, 0.01, 0.1, 1, 10, 100, 1000]
    'gamma': np.logspace(-3, 3, 5)
}

# Creazione del modello SVM
svm = SVC()

# GridSearchCV con cv=10
svm_cv = GridSearchCV(svm, parameters, cv=10)

# Fit sul training set
svm_cv.fit(X_train, Y_train)

# Migliori parametri trovati
print("Best parameters:", svm_cv.best_params_)

# Miglior punteggio di cross-validation
print("Best cross-validation score:", svm_cv.best_score_)

# Accuracy sul test set
print("Test set accuracy:", svm_cv.score(X_test, Y_test))


Best parameters: {'C': np.float64(1000.0), 'gamma': np.float64(0.03162277660168379), 'kernel': 'sigmoid'}
Best cross-validation score: 0.8321428571428571
Test set accuracy: 0.7777777777777778


In [ ]:
from sklearn.tree import DecisionTreeClassifier

# ============ TASK 8 ============

# Definizione del dizionario dei parametri
parameters = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [2*n for n in range(1,10)],
    'max_features': ['sqrt', 'log2'],  # 'auto' è deprecato per DecisionTreeClassifier
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10]
}

# Creiamo il modello Decision Tree
tree = DecisionTreeClassifier(random_state=2)

# Creiamo il GridSearchCV con 10-fold cross-validation
tree_cv = GridSearchCV(tree, parameters, cv=10)

# Fit sul training set
tree_cv.fit(X_train, Y_train)

# Migliori parametri trovati
print("Best parameters:", tree_cv.best_params_)

# Accuracy media in cross-validation
print("Best cross-validation score:", tree_cv.best_score_)

# Accuracy sul test set
print("Test set accuracy:", tree_cv.score(X_test, Y_test))


Best parameters: {'criterion': 'gini', 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'splitter': 'random'}
Best cross-validation score: 0.8160714285714284
Test set accuracy: 0.8888888888888888


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# ============ TASK 10 ============

# Dizionario dei parametri da testare
parameters = {
    'n_neighbors': [1,2,3,4,5,6,7,8,9,10],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'p': [1, 2]   # p=1 -> Manhattan, p=2 -> Euclidean
}

# Creazione del modello KNN
KNN = KNeighborsClassifier()

# GridSearchCV con 10-fold cross validation
knn_cv = GridSearchCV(KNN, parameters, cv=10)

# Fit sul training set
knn_cv.fit(X_train, Y_train)

# Migliori parametri trovati
print("Best parameters:", knn_cv.best_params_)

# Migliore accuracy media in cross-validation
print("Best cross-validation score:", knn_cv.best_score_)

# Accuracy sul test set
print("Test set accuracy:", knn_cv.score(X_test, Y_test))


Best parameters: {'algorithm': 'auto', 'n_neighbors': 3, 'p': 2}
Best cross-validation score: 0.7785714285714286
Test set accuracy: 0.7222222222222222


In [ ]:
# ========TASK12=======
#===== Riepilogo delle accuracy ============
# Accuracy di ciascun modello
logreg_accuracy = logreg_cv.score(X_test, Y_test)
svm_accuracy = svm_cv.score(X_test, Y_test)
knn_accuracy = knn_cv.score(X_test, Y_test)

# Creiamo un DataFrame per confrontarle facilmente
import pandas as pd

accuracy_summary = pd.DataFrame({
    'Model': ['Logistic Regression', 'SVM', 'KNN'],
    'Test Accuracy': [logreg_accuracy, svm_accuracy, knn_accuracy]
})

# Mostriamo il risultato in percentuale
accuracy_summary['Test Accuracy (%)'] = accuracy_summary['Test Accuracy'] * 100

print(accuracy_summary)


                 Model  Test Accuracy  Test Accuracy (%)
0  Logistic Regression       0.833333          83.333333
1                  SVM       0.777778          77.777778
2                  KNN       0.722222          72.222222


In [ ]:
print(Y)

[0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 1 0 0 1 1 1 1 1 0 1 1 0 1 1 0 1 1 1 0 1 1
 1 1 1 1 1 1 1 1 0 0 0 1 1 0 0 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 0 1
 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1]
